In [1]:
!pip install huggingface_hub

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install auto-gptq
!pip install git+https://github.com/huggingface/optimum.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/optimum.git to /tmp/pip-req-build-y9oauf2p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/optimum.git /tmp/pip-req-build-y9oauf2p
  Resolved https://github.com/huggingface/optimum.git to commit fd47a73267c3a71ea4e3c02f92260ae61c5ae372
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

model_id = "TheBloke/Llama-2-7B-Chat-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_id)
gptq_config = quantization_config = GPTQConfig(
    bits=4,
    dataset="c4",
    tokenizer=tokenizer,
    group_size=128,
    model_seqlen=2048
)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [5]:
quantized_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=gptq_config)

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4199: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [19]:
prompt = 'pizza vs farmers pie'
tokens = tokenizer.encode(prompt, return_tensors='pt')
tokens = tokens.to('cuda')
output = quantized_model.generate(tokens, num_return_sequences=1)

In [20]:
gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(gen_text)

pizza vs farmers pie

Pizza and farmers pie are two popular dishes that are often compared and contrasted. Here are some key differences between the two:

Pizza:

* Pizza is a flatbread topped with ingredients such as cheese, tomato sauce, and various meats or vegetables.
* It is typically baked in an oven until the crust is crispy and the toppings are heated through.
* Pizza originated in Italy and has since become a popular dish around the world.
* Pizza can be served as a main course or as a snack.

Farmers Pie:

* Farmers pie is a savory pie filled with ingredients such as beef, vegetables, and gravy.
* It is typically made with a flaky pastry crust and baked until the filling is hot and the crust is golden brown.
* Farmers pie originated in the United Kingdom and is often served as a comfort food or a hearty meal.
* Farmers pie can be served as a main course or as a side dish.

In summary, pizza and farmers pie are two different dishes with distinct ingredients and cooking methods

In [21]:
name = "vXofi/Llama-2-7B-chat-GPTQ-to-GPTQ4"
quantized_model.push_to_hub(name)
tokenizer.push_to_hub(name)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 4096}


model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vXofi/Llama-2-7B-chat-GPTQ-to-GPTQ4/commit/1823c52e79c5a17b19454d095b6943b8b3574fbc', commit_message='Upload tokenizer', commit_description='', oid='1823c52e79c5a17b19454d095b6943b8b3574fbc', pr_url=None, pr_revision=None, pr_num=None)